# Ανίχνευση του λιπώδους ιστού με συσταδοποίηση
## Εντοπισμός της συμπεριφοράς της θερμογένεσης.
## Η καταγραφή για εννέα δείγματα αφού προηγηθεί segmentation στα πέντε ποντίκια του πειράματος
### Επιλογή εικόνων για τη μέτρηση της μέσης θερμοκρασίας και της τυπικής απόκλισης
Στο αρχείο bat_list.yml έχουμε ορίσει από ποιες εικόνες θα καταγράψουμε τη μέση θερμοκρασία ενός τετραγωνικού patch το οποίο βρίσκεται στον αυχένα ενός ποντικιού, όπως φαίνεται στην παραπάνω εικόνα. 

Δομή του αρχείου yaml: 
```
samples: [0h, 24h, 48h, 72h, 96h, 120h, 144h, 192h, 240h]
mouse_id: [mouse_1, mouse_2, mouse_3, mouse_4, mouse_5]
mouse_1:
  csv: [CSV_2060.csv, CSV_2305.csv, CSV_2509.csv, CSV_2730.csv, CSV_2957.csv, CSV_3181.csv, CSV_3397.csv, CSV_3652.csv, CSV_3855.csv]
  jpeg: [IR_2060.jpg, IR_2305.jpg, IR_2509.jpg, IR_2730.jpg, IR_2957.jpg, IR_3181.jpg, IR_3397.jpg, IR_3652.jpg, IR_3855.jpg]
.
.
. κλπ
```

In [1]:
import os
import sys
import yaml
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from celluloid import Camera
from IPython.display import HTML # to show the animation in Jupyter
from tqdm import tqdm

from skimage.exposure import histogram
from skimage.filters.rank import entropy
from skimage.morphology import disk, square, skeletonize, thin, dilation
from skimage.util import img_as_ubyte
from skimage.feature import greycomatrix, greycoprops, local_binary_pattern
from sklearn import cluster
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from src.features import img_to_vectors
from src.features import mouse_detection
from src.data import data_utils
from sklearn import preprocessing

# Εισαγωγή των δεδομένων μέσω yaml
with open(module_path + '/data/bat_list.yml') as file:
    experiment = yaml.load(file, Loader=yaml.FullLoader)
# access: experiment['samples'], experiment['mouse_id']

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

src.data.data_utils


### Κατασκευή global path για τα δεδομένα και απομάκρυνση background 

In [10]:
# Πρόσβαση στα δεδομένα του αρχείου
mouse_names = experiment['mouse_id']
# mouse_names = ['mouse_1']
# Κατασκευή των global path για τα αρχεία του πειράματος
infrared_items = []
csv_items = []
for mouse_name in mouse_names:
    for hour, jpeg_file in zip(experiment['samples'], experiment[mouse_name]['jpeg']):
        full_jpeg_path = module_path + '/data/raw/' + hour + '/' + mouse_name + '/' + jpeg_file
        infrared_items.append(full_jpeg_path)

    for hour, csv_file in zip(experiment['samples'], experiment[mouse_name]['csv']):
        full_csv_path = module_path + '/data/raw/' + hour + '/' + mouse_name + '/' + csv_file
        csv_items.append(full_csv_path)
# Εισαγωγή των δεδομένων με βάση τα path και οι προεπεξεργασίες τους
infrared_images = [] # will store (238, 98), (229, 100) arrays etc
exif_data = []
for item in infrared_items:
    infrared_image, exif_datum = data_utils.IR_fixer(fpath=item)
    infrared_images.append(infrared_image)
    exif_data.append(exif_datum)

markerBack, markerBody = 70, 150
object_images, object_masks, initial_masks, mouse_locations = mouse_detection.mouse_detection(infrared_images, markerBack, markerBody)